<a href="https://colab.research.google.com/github/farahbakhsh3/DeepAtariLearning/blob/master/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://www.dropbox.com/s/nxz7aqtmsiusezd/atari.rar
!apt install unrar
!unrar x -r atari.rar


--2018-10-30 13:50:16--  https://www.dropbox.com/s/nxz7aqtmsiusezd/atari.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nxz7aqtmsiusezd/atari.rar [following]
--2018-10-30 13:50:16--  https://www.dropbox.com/s/raw/nxz7aqtmsiusezd/atari.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95c8aa2c12549abb75bf7517f0.dl.dropboxusercontent.com/cd/0/inline/AUUjkym8oE2O2ecVhGHHniBY4WZHTdFsUKBOjM7d9CV4_t6YTXzMTRjmqsSzY6MyPdAswSMJ5-h9DleEY5oSfqH8HfHycxXrJlmYNfA_r6GQwemRQLDCUiGOjUNWfrG0Kyp603zFcyEop1TdaWod8oTqXdPzthLw9S3qJcNmB0U2vN9erAmGO5cYYBvES2Fte7U/file [following]
--2018-10-30 13:50:16--  https://uc95c8aa2c12549abb75bf7517f0.dl.dropboxusercontent.com/cd/0/inline/AUUjkym8oE2O2ecVhGHHniBY4WZHTdFsUKBOjM7d9CV4_t6YTXzMTRjmq

In [0]:
#!/usr/bin/env python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, ConvLSTM2D, MaxPool2D
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from prepare import Sample, model_def
import matplotlib.pyplot as plt


def create_model_ConvLSTM2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):
    
    model = Sequential()

    model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, input_shape=INPUT_SHAPE))
    model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True))
    model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same',  
                            activation='relu', return_sequences=True))
    model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same', 
                            activation='relu', strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    print(model.summary())

    return model


def create_model_Conv2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):

    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu', input_shape=INPUT_SHAPE))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    
    print(model.summary())
    
    return model

def convert_data_to_series(x_train, y_train, window_size):
    x_ = []
    for i in range(x_train.shape[0] - window_size):
        x_.append(x_train[i:i + window_size])
    x_train = np.array(x_)
    y_train = y_train[:x_train.shape[0]]

    return x_train, y_train


if __name__ == '__main__':

    _model_def = model_def()
    x_train = np.load("X.npy")
    y_train = np.load("Y.npy")
    y_train = to_categorical(y_train, num_classes=_model_def.OUT_SHAPE)

    epochs = 50
    batch_size = 128

    if _model_def.model == 'Conv2D':
        model = create_model_Conv2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)
    elif _model_def.model == 'ConvLSTM2D':
        x_train, y_train = convert_data_to_series(x_train[:100], y_train[:100], 15)
        model = create_model_ConvLSTM2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)

    print('X_Train Shape: ', x_train.shape)
    print('Y_Train Shape: ', y_train.shape)
    print('---------------------------')
    
    if input('Train from zero: <z>  ,  Retrain by load prev weights: <r>  ::   ') == 'r':
        model.load_weights(_model_def.weights_file)
        print('Model weights loaded : ', _model_def.weights_file) 
    else:
        print('Train from zero.')
    print('---------------------------')
        
    chekpoint = ModelCheckpoint(filepath=_model_def.weights_file, 
                                monitor='val_acc', 
                                save_best_only=True, 
                                save_weights_only=True, 
                                verbose=True)
    
    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        shuffle=True, 
                        validation_split=0.05, 
                        callbacks=[chekpoint])

    # model.save_weights(_model_def.weights_file)
    print('Best weights saved. ', _model_def.weights_file)
    print('---------------------------')


    history_dict = history.history
    loss_value = history_dict['acc']
    val_loss_value = history_dict['val_acc']
    epoches = range(1,len(loss_value) + 1)

    plt.plot(epoches, loss_value, 'bo', label='Traning accuracy')
    plt.plot(epoches, val_loss_value, 'b', label='Validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epoches')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

1: Boxing
2: RiverRaid
3: Enduro
4: SeaQuest
5: Tennis
6: SpaceInvaders


In [2]:
x_train.shape

(100, 5, 80, 3)